## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [1]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 
!pip install llama-cpp-python

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 19.5 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 28.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp313-cp313-macosx_15_0_arm64.whl size=3503698 sha256=daa0efbef30e75ff74e25180c77260a5fb1935af2415cee7740e361c4cf3106c
  Stored in directory: /Users/rudraprakashpandey/Library/Caches/pip/wheels/17/34/ea/37b6118820f9f8ac2a27193fcaa2c5cb95f92ffe2c7f808540
Successfully built llama-cpp-python

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install numpy
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 22.7 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# For installing the libraries & downloading models from HF Hub
%pip install huggingface_hub
%pip install tiktoken
%pip install pymupdf
%pip install langchain
%pip install langchain-community
%pip install chromadb
%pip install sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 20.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.9 MB/s et

In [24]:
#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [25]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"


model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

print(f"Model downloaded to: {model_path}")

llm = Llama(
    model_path=model_path,
    n_ctx=2300,
    n_gpu_layers=8,
    n_batch=128
)


llama_model_load_from_file_impl: using device Metal (Apple M4) - 9434 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/rudraprakashpandey/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_load

Model downloaded to: /Users/rudraprakashpandey/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf


ggml_backend_metal_log_allocated_size: allocated buffer, size =  1365.27 MiB, ( 2853.62 / 10922.67)
load_tensors: offloading 8 repeating layers to GPU
load_tensors: offloaded 8/33 layers to GPU
load_tensors: Metal_Mapped model buffer size =  1365.26 MiB
load_tensors:   CPU_Mapped model buffer size =  5666.09 MiB
...................................................................................................
llama_context: constructing llama_context
llama_context: n_seq_max     = 1
llama_context: n_ctx         = 2300
llama_context: n_ctx_per_seq = 2300
llama_context: n_batch       = 128
llama_context: n_ubatch      = 128
llama_context: causal_attn   = 1
llama_context: flash_attn    = 0
llama_context: freq_base     = 1000000.0
llama_context: freq_scale    = 1
llama_context: n_ctx_per_seq (2300) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
ggml_metal_init: allocating
ggml_metal_init: found device: Apple M4
ggml_metal_init: picking default device: Apple M

#### Response

In [26]:
# Temparature is kept at 0 because for this business use case we want factual answers

def response(query,max_tokens=200,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [27]:
query1 = "What is the protocol for managing sepsis in a critical care unit?"
print(response(query1))


llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =     711.25 ms /    16 tokens (   44.45 ms per token,    22.50 tokens per second)
llama_perf_context_print:        eval time =   15773.26 ms /   199 runs   (   79.26 ms per token,    12.62 tokens per second)
llama_perf_context_print:       total time =   16510.03 ms /   215 tokens




Sepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are the general steps for managing sepsis in a critical care unit:

1. Early recognition and suspicion: Septic patients may present with non-specific symptoms such as fever, chills, tachycardia, tachypnea, altered mental status, and lactic acidosis. It is essential to have a high index of suspicion for sepsis, especially in patients with known infections or risk factors.
2. Initial assessment and resuscitation: The first step in managing sepsis is to assess and resuscitate the patient. This includes assessing airway, breathing, circulation, and disability (ABCD) and providing appropriate interventions such as oxygen therapy, fluid resuscitation, and vasopressor support as needed.
3. Source control:


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [9]:
query2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
print(response(query2))

Llama.generate: 2 prefix-match hit, remaining 32 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1522.40 ms /    32 tokens (   47.58 ms per token,    21.02 tokens per second)
llama_perf_context_print:        eval time =   17317.64 ms /   199 runs   (   87.02 ms per token,    11.49 tokens per second)
llama_perf_context_print:       total time =   18868.19 ms /   231 tokens




Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The symptoms of appendicitis can vary from person to person, but the following are the most common:

1. Abdominal pain: The pain is typically located in the lower right side of the abdomen and may start as a mild discomfort that gradually worsens. The pain may be constant or come and go, and it may be accompanied by cramping or bloating.
2. Loss of appetite: People with appendicitis may lose their appetite and feel nauseous or vomit.
3. Fever: A fever of 100.4°F (38°C) or higher is common in appendicitis.
4. Diarrhea or constipation: Some people with appendicitis may experience diarrhea


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [10]:
query3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
print(response(query3))

Llama.generate: 4 prefix-match hit, remaining 34 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1665.91 ms /    34 tokens (   49.00 ms per token,    20.41 tokens per second)
llama_perf_context_print:        eval time =   16450.18 ms /   199 runs   (   82.66 ms per token,    12.10 tokens per second)
llama_perf_context_print:       total time =   18142.05 ms /   233 tokens




Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles, leading to hair loss in small, round patches on the scalp, beard, or other areas of the body. The exact cause of alopecia areata is not known, but it is believed to be related to a combination of genetic and environmental factors that trigger an abnormal immune response.

There are several treatments and solutions that have been shown to be effective in addressing sudden patchy hair loss:

1. Corticosteroids: Corticosteroids are anti-inflammatory medications that can help reduce inflammation and suppress the immune system, allowing the hair follicles to regrow. They can be applied topically or taken orally, depending on the severity and extent of the hair loss.
2. Immunotherapy: Immunotherapy involves the use of medications


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [11]:
query4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
print(response(query4))

Llama.generate: 2 prefix-match hit, remaining 28 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1066.98 ms /    28 tokens (   38.11 ms per token,    26.24 tokens per second)
llama_perf_context_print:        eval time =   17199.60 ms /   199 runs   (   86.43 ms per token,    11.57 tokens per second)
llama_perf_context_print:       total time =   18293.63 ms /   227 tokens




There is no one-size-fits-all answer to this question, as the specific treatment recommendations for a person with a brain injury depend on the severity and location of the injury, as well as the individual's age, overall health, and other factors. However, there are some common treatments and interventions that may be recommended for individuals with brain injuries.

1. Acute care: In the immediate aftermath of a brain injury, the focus is on providing acute care to stabilize the person's condition and prevent further damage. This may include:

- Emergency medical care: If the brain injury is severe, the person may require emergency medical care, such as surgery to remove a hematoma or decompress a skull fracture.
- Medications: Depending on the specific symptoms of the brain injury, the person may be prescribed medications to manage symptoms such as pain, swelling, or seizures.
- Re


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [12]:
query5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
print(response(query5))

Llama.generate: 2 prefix-match hit, remaining 35 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1548.40 ms /    35 tokens (   44.24 ms per token,    22.60 tokens per second)
llama_perf_context_print:        eval time =   18090.06 ms /   199 runs   (   90.90 ms per token,    11.00 tokens per second)
llama_perf_context_print:       total time =   19672.86 ms /   234 tokens




First and foremost, if a person has fractured their leg during a hiking trip, it is essential to ensure their safety and prevent further injury. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the extent of the injury and assess the person's condition. If the fracture is open or the person is in severe pain, immobilize the leg with a splint or a makeshift sling to prevent any movement.
2. Call for help: If possible, call for emergency medical assistance. If there is no cell phone reception, try to signal for help using a mirror, whistle, or other means.
3. Provide first aid: Apply a sterile dressing to the injury to prevent infection. If the fracture is open, apply a clean cloth to stop the bleeding.
4. Immobilize the leg: Use a splint, a makeshift sling, or


## Question Answering using LLM with Prompt Engineering

In [28]:
# Add instructions to the prompt for better response generation
# Given the sensitivity of medical diagnosis, I am adding a prompt to minimize hallucination
system_prompt = "You are a helpful and knowledgeable medical assistant. Answer the following medical question accurately and concisely based on common medical knowledge. If you don't know the answer, please state that you don't have enough information."
print ("System Prompt:" + system_prompt)

System Prompt:You are a helpful and knowledgeable medical assistant. Answer the following medical question accurately and concisely based on common medical knowledge. If you don't know the answer, please state that you don't have enough information.


### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [29]:
query1 = system_prompt + "\n" + "What is the protocol for managing sepsis in a critical care unit?"
print(response(query1))


Llama.generate: 1 prefix-match hit, remaining 61 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =    1571.70 ms /    61 tokens (   25.77 ms per token,    38.81 tokens per second)
llama_perf_context_print:        eval time =   15922.62 ms /   199 runs   (   80.01 ms per token,    12.50 tokens per second)
llama_perf_context_print:       total time =   17520.87 ms /   260 tokens



Sepsis is a life-threatening condition caused by a severe infection. In a critical care unit, managing sepsis involves the following steps:
1. Early recognition and diagnosis: Identify sepsis early based on clinical signs and laboratory results, such as fever, tachycardia, tachypnea, low blood pressure, and elevated white blood cell count.
2. Immediate fluid resuscitation: Administer intravenous fluids to maintain adequate blood pressure and organ perfusion.
3. Antibiotic therapy: Start broad-spectrum antibiotics as soon as possible based on the suspected infection site and microbiological culture results.
4. Source control: Identify and address the source of infection, such as removing an infected catheter or draining an abscess.
5. Vasopressor support: If the patient's blood pressure remains low despite fluid resuscitation, administer vas


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [15]:
query2 = system_prompt + "\n" + "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
print(response(query2))

Llama.generate: 48 prefix-match hit, remaining 32 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1449.14 ms /    32 tokens (   45.29 ms per token,    22.08 tokens per second)
llama_perf_context_print:        eval time =   17215.40 ms /   199 runs   (   86.51 ms per token,    11.56 tokens per second)
llama_perf_context_print:       total time =   18693.23 ms /   231 tokens




Appendicitis is a common inflammatory condition of the appendix, a small tube-shaped organ located in the lower right abdomen. The symptoms of appendicitis can include:

1. Sudden pain in the lower right abdomen, which may start as a mild ache and gradually develop into a sharp pain.
2. Loss of appetite and feeling sick to your stomach (nausea).
3. Fever, which may be low-grade at first but can rise as high as 101°F (38.3°C) or higher.
4. Vomiting, which may help relieve abdominal pain.
5. Constipation or diarrhea.
6. Inability to pass gas or have a bowel movement.
7. Pain in the lower right quadrant of the abdomen when the doctor presses on it during a physical exam.




### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [16]:
query3 = system_prompt + "\n" + "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
print(response(query3))

Llama.generate: 50 prefix-match hit, remaining 34 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1687.04 ms /    34 tokens (   49.62 ms per token,    20.15 tokens per second)
llama_perf_context_print:        eval time =   16727.44 ms /   199 runs   (   84.06 ms per token,    11.90 tokens per second)
llama_perf_context_print:       total time =   18440.97 ms /   233 tokens




Sudden patchy hair loss, also known as alopecia areata, is an autoimmune condition that causes hair loss in small, round patches on the scalp, beard, or other areas of the body. The exact cause is unknown, but it's believed to be related to a problem with the immune system.

Effective treatments for addressing sudden patchy hair loss include:

1. Corticosteroids: These are anti-inflammatory medications that can help reduce inflammation and suppress the immune system response. They can be applied topically or taken orally.
2. Immunotherapy: This involves the use of medications that stimulate the immune system to attack the hair loss. One such medication is minoxidil.
3. Hair transplantation: This is a surgical procedure in which healthy hair is transplanted from one area of the scalp to another. It's usually considered a


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [17]:
query4 = system_prompt + "\n" + "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
print(response(query4))

Llama.generate: 48 prefix-match hit, remaining 28 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1078.00 ms /    28 tokens (   38.50 ms per token,    25.97 tokens per second)
llama_perf_context_print:        eval time =   17183.78 ms /   199 runs   (   86.35 ms per token,    11.58 tokens per second)
llama_perf_context_print:       total time =   18287.81 ms /   227 tokens




For a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, the following treatments may be recommended based on common medical knowledge:

1. Emergency care: If the injury is recent, the person may require emergency care to address any life-threatening conditions, such as airway obstruction, breathing difficulties, or excessive bleeding.
2. Medications: Depending on the specific symptoms and conditions, various medications may be prescribed to manage symptoms, prevent complications, or improve brain function. For example, anti-inflammatory drugs may be used to reduce swelling, anticonvulsants to prevent seizures, or stimulants to improve attention and focus.
3. Rehabilitation: Rehabilitation programs, including physical, occupational, and speech therapy, can help individuals regain lost skills and improve overall function. These therapies may focus on areas such as mobility,


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [18]:
query5 = system_prompt + "\n" + "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
print(response(query5))

Llama.generate: 48 prefix-match hit, remaining 35 prompt tokens to eval
llama_perf_context_print:        load time =    3113.49 ms
llama_perf_context_print: prompt eval time =    1577.05 ms /    35 tokens (   45.06 ms per token,    22.19 tokens per second)
llama_perf_context_print:        eval time =   17460.75 ms /   199 runs   (   87.74 ms per token,    11.40 tokens per second)
llama_perf_context_print:       total time =   19069.61 ms /   234 tokens




A leg fracture during a hiking trip requires prompt medical attention. Here are the necessary precautions and treatment steps:

1. Immobilize the fracture: Use a splint or a sling to immobilize the affected leg to prevent further damage and pain. If the fracture is severe, do not move the person unless it is necessary to get them to medical help.
2. Control bleeding: Apply direct pressure to the wound to control bleeding. If the bleeding does not stop, apply a sterile dressing and elevate the leg above heart level.
3. Pain relief: Provide pain relief using over-the-counter pain medications such as acetaminophen or ibuprofen. If the pain is severe, the person may need prescription pain medication.
4. Transport to medical help: Arrange for transportation to the nearest medical facility as soon as possible. If the person cannot be moved, call for emergency medical services.


## Data Preparation for RAG

### Loading the Data

In [31]:
## Data Preparation for RAG
### Loading the Data
#Libraries for processing dataframes, text
import json, os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

### Data Overview

In [32]:
## Data Overview


pdf_path = '/Users/rudraprakashpandey/Documents/code/Vscode/medical/medical_diagnosis_manual (1).pdf'
if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"PDF not found at {pdf_path}")

# Use the PyMuPDFLoader to load the document
try:
    # Note: Loading large PDFs (>4,000 pages) may require significant memory. Consider chunked processing if RAM is limited.
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from the PDF.")
except Exception as e:
    print(f"Error loading PDF: {e}")
    raise

Loaded 4114 pages from the PDF.


#### Checking the first 5 pages

In [33]:
# Preview first 5 pages (or fewer if PDF is smaller) for debugging
for i in range(min(5, len(documents))):
    print(f"--- Page {i+1} ---")
    print(documents[i].page_content[:500] + "...")

#### Checking the number of pages
print(f"Number of pages: {len(documents)}")

--- Page 1 ---
vistarathecompany@gmail.com
6LVJBSDN4X
for personal use by vistarathecompany@
shing the contents in part or full is liable...
--- Page 2 ---
vistarathecompany@gmail.com
6LVJBSDN4X
This file is meant for personal use by vistarathecompany@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action....
--- Page 3 ---
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ....................................
--- Page 4 ---
491
Chapter 44. Foot & Ankle Disorders    ..........................................................................

#### Checking the number of pages

### Data Chunking

In [34]:
## Function to do data chunking

def get_data_chunks(
    data,
    chunk_size=1000,
    chunk_overlap=200,
    split_method="recursive",
    separators=["\n\n", "\n", ". ", " ", ""],
    min_chunk_size=50,
    respect_sentence_boundaries=True,
    respect_paragraph_boundaries=True,
    length_function=len,
    max_chunks=None,
    add_metadata=True
):
    """
    Splits a list of documents into smaller chunks using a specified method.

    Args:
        data: A list of document objects (e.g., from Langchain loaders).
        chunk_size: The maximum size of each chunk.
        chunk_overlap: The number of characters to overlap between chunks.
        split_method: The method to use for splitting ('recursive').
        separators: A list of separators to use for splitting.
        min_chunk_size: The minimum size of each chunk.
        respect_sentence_boundaries: Whether to try to split on sentence boundaries.
        respect_paragraph_boundaries: Whether to try to split on paragraph boundaries.
        length_function: The function to use to measure chunk length.
        max_chunks: The maximum number of chunks to generate.
        add_metadata: Whether to add metadata to the chunks.

    Returns:
        A list of chunked documents.
    """
    try:
        if split_method == "recursive":
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                length_function=length_function,
                is_separator_regex=False,
                separators=separators
            )
            chunks = text_splitter.split_documents(data)
        else:
            raise ValueError(f"Unsupported split_method: {split_method}")
    except Exception as e:
        print(f"Error during chunking: {e}")
        raise
    chunks = [chunk for chunk in chunks if length_function(chunk.page_content) >= min_chunk_size]
    if add_metadata:
        for chunk in chunks:
            if not chunk.metadata:
                chunk.metadata = {"source": "medical_diagnosis_manual.pdf", "page": 0}  # Fallback
    if max_chunks is not None:
        chunks = chunks[:max_chunks]
    return chunks

In [35]:

# Utilize the get_data_chunks function for the loaded PDF
chunk_size = 500  # chunk size
chunk_overlap = 100 # chunk overlap

chunks = get_data_chunks(
    documents,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    split_method="recursive",
    separators=["\n\n", "\n", ". ", " ", ""],
    min_chunk_size=50,
    respect_sentence_boundaries=True,
    respect_paragraph_boundaries=True,
    length_function=len,
    max_chunks=None, # Process all chunks
    add_metadata=True
)

# Print some validation and check statements
print(f"\n--- Chunking Validation ---")
print(f"Number of documents loaded: {len(documents)}")
print(f"Number of chunks created: {len(chunks)}")

# Check the first few chunks
print(f"\nFirst {min(5, len(chunks))} chunks:")
for i in range(min(5, len(chunks))):
  print(f"--- Chunk {i+1} ---")
  print(f"Chunk length: {len(chunks[i].page_content)}")
  print(f"Chunk metadata: {chunks[i].metadata}")
  print(chunks[i].page_content[:200] + "...") # Print first 200 characters of the chunk content

# Check the last few chunks (if there are more than 5)
if len(chunks) > 5:
    print(f"\nLast {min(5, len(chunks)-5)} chunks:")
    for i in range(max(0, len(chunks)-5), len(chunks)):
        print(f"--- Chunk {i+1} ---")
        print(f"Chunk length: {len(chunks[i].page_content)}")
        print(f"Chunk metadata: {chunks[i].metadata}")
        print(chunks[i].page_content[:200] + "...") # Print first 200 characters of the chunk content

# Additional checks
if len(chunks) > 0:
    # Check minimum chunk size
    min_len = min(len(chunk.page_content) for chunk in chunks)
    print(f"\nMinimum chunk length: {min_len}")
    if min_len < 50: # Based on min_chunk_size parameter
        print("Warning: Some chunks might be smaller than the specified minimum size.")

    # Check for empty chunks
    empty_chunks = sum(1 for chunk in chunks if len(chunk.page_content) == 0)
    print(f"Number of empty chunks: {empty_chunks}")

    # Check metadata presence (assuming add_metadata is True)
    metadata_missing = sum(1 for chunk in chunks if not hasattr(chunk, 'metadata') or not chunk.metadata)
    print(f"Number of chunks missing metadata: {metadata_missing}")
else:
    print("\nNo chunks were created.")

ggml_metal_free: deallocating
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)
ggml_metal_mem_pool_free: freeing memory pool, num heaps = 0 (total = 0)



--- Chunking Validation ---
Number of documents loaded: 4114
Number of chunks created: 34605

First 5 chunks:
--- Chunk 1 ---
Chunk length: 122
Chunk metadata: {'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': '/Users/rudraprakashpandey/Documents/code/Vscode/medical/medical_diagnosis_manual (1).pdf', 'file_path': '/Users/rudraprakashpandey/Documents/code/Vscode/medical/medical_diagnosis_manual (1).pdf', 'total_pages': 4114, 'format': 'PDF 1.7', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-05-13T02:35:28+00:00', 'trapped': '', 'modDate': 'D:20250513023528Z', 'creationDate': 'D:20120615054440Z', 'page': 0}
vistarathecompany@gmail.com
6LVJBSDN4X
for personal use by vistarathecompany@
shing the contents in part or full is liable...
--- Chunk 2 ---
Chunk length: 190
Chunk metadata: {'producer': 'pdf-lib 

### Embedding

In [36]:
# Note: For M4 with 16GB RAM, process chunks in small batches to avoid memory issues.
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Embed chunks in batches
embedded_chunks = []
batch_size = 100  # Safe for 16GB RAM
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i+batch_size]
    for j, chunk in enumerate(batch):
        try:
            embedding = embedding_function.embed_query(chunk.page_content)
            embedded_chunks.append((chunk, embedding))
        except Exception as e:
            print(f"Error embedding chunk {i+j}: {e}")
print(f"\n--- Embedding Validation ---")
print(f"Number of original chunks: {len(chunks)}")
print(f"Number of successfully embedded chunks: {len(embedded_chunks)}")
if len(embedded_chunks) == len(chunks):
    print("All chunks successfully embedded.")
else:
    print(f"Embedded {len(embedded_chunks)}/{len(chunks)} chunks.")

if len(embedded_chunks) > 0:
    first_embedded_chunk, first_embedding = embedded_chunks[0]
    print(f"Type of first embedding: {type(first_embedding)}")
    import numpy as np
    first_embedding_np = np.array(first_embedding)
    print(f"Dimension of first embedding: {len(first_embedding_np)}")
    print(f"First 10 values: {first_embedding_np[:10]}")

/var/folders/2y/flkwlsc160n2tz5m70f_nt3m0000gn/T/ipykernel_1050/3405431599.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



--- Embedding Validation ---
Number of original chunks: 34605
Number of successfully embedded chunks: 34605
All chunks successfully embedded.
Type of first embedding: <class 'list'>
Dimension of first embedding: 384
First 10 values: [-0.07890099  0.06623081  0.03226828 -0.04527882  0.03095172  0.01784509
  0.03430261  0.01443451 -0.00089051 -0.02856948]


### Vector Database

In [37]:
#Vector Database
import os, shutil

from langchain_community.vectorstores import Chroma

persist_directory = 'medical_db'
if os.path.exists(persist_directory):
    print(f"Removing existing database at {persist_directory}")
    shutil.rmtree(persist_directory, ignore_errors=True)

if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)
    print(f"Created directory at {persist_directory}")

try:
    vector_db = Chroma.from_documents(
        documents=[chunk for chunk, embedding in embedded_chunks],
        embedding=embedding_function,
        persist_directory=persist_directory
    )
except Exception as e:
    print(f"Error creating Chroma database: {e}")
    raise

# Simplified vector database validation
print(f"\n--- Vector Database Validation ---")
if os.path.exists(persist_directory):
    print(f"Chroma database created at {persist_directory}.")
try:
    count = vector_db._collection.count()
    print(f"Number of items in database: {count}")
    print("Match with embedded chunks." if count == len(embedded_chunks) else "Item count mismatch.")
except Exception as e:
    print(f"Error retrieving database count: {e}")

Created directory at medical_db

--- Vector Database Validation ---
Chroma database created at medical_db.
Number of items in database: 34605
Match with embedded chunks.


### Retriever

In [39]:
# prompt: code a retriever using the above code with the appropriate search method and k value
retriever = vector_db.as_retriever(
    search_type="similarity",  # Using similarity search
    search_kwargs={"k": 3}     # Retrieve top 3 similar documents
)

# --- Validation and Conclusions ---
print(f"\n--- Retriever Validation ---")
if retriever:
    print("Conclusion: Retriever successfully created.")
    try:
        # Test with queries from problem statement (e.g., sepsis, appendicitis, hair loss)
        sample_query = "What are the symptoms of appendicitis?"
        retrieved_docs = retriever.invoke(sample_query)
        print(f"\nRetrieved {len(retrieved_docs)} documents for a sample query.")
        if len(retrieved_docs) > 0:
            print("Sample of retrieved document content (first 100 chars):")
            print(retrieved_docs[0].page_content[:100] + "...")
        else:
            print("No documents retrieved. Check vector database content or query relevance.")
        
        print(f"\nRetriever configuration:")
        print(f"Search type: {retriever.search_type}")
        print(f"Search kwargs: {retriever.search_kwargs}")
        if retriever.search_kwargs.get("k") == 3:
            print("Conclusion: Retriever configured with correct k value (3).")
        else:
            print(f"Warning: Retriever's k value is {retriever.search_kwargs.get('k')}, expected 3.")
    except ValueError as ve:
        print(f"Database error during retrieval: {ve}")
    except Exception as e:
        print(f"Error during sample query with retriever: {e}")
        print("Conclusion: Retriever might not be configured correctly or database has issues.")
else:
    print("Conclusion: Failed to create the retriever.")


--- Retriever Validation ---
Conclusion: Retriever successfully created.

Retrieved 3 documents for a sample query.
Sample of retrieved document content (first 100 chars):
Symptoms and Signs
The classic symptoms of acute appendicitis are epigastric or periumbilical pain f...

Retriever configuration:
Search type: similarity
Search kwargs: {'k': 3}
Conclusion: Retriever configured with correct k value (3).


### System and User Prompt Template

In [40]:
## 1. The system message describing the assistant's role.
## 2. A user message template including context and the question.

In [41]:
# --- System and User Prompts ---
qna_system_message = "You are a knowledgeable medical assistant. Provide accurate, concise answers based solely on the provided context from the Merck Manuals. If the context is insufficient, state that you lack information."
qna_user_message_template = """Context: {context}\n\nQuestion: {question}\nAnswer concisely and factually."""


In [42]:
# --- Evaluation Prompts ---
groundedness_rater_system_message = "You are an evaluator assessing the groundedness of a medical response. Rate the response based on whether all factual claims are supported by the provided context, on a scale of 1–5 (1: contradicts context, 5: fully supported). Return only the numeric rating."
relevance_rater_system_message = "You are an evaluator assessing the relevance of a medical response. Rate the response based on how directly it addresses the question, on a scale of 1–5 (1: irrelevant, 5: fully relevant). Return only the numeric rating."
user_message_template = """Question: {question}\nResponse: {answer}\nContext: {context}\nRating:"""


In [43]:
# --- Queries to Test ---
queries_to_test = {
    "Query 1": "What is the protocol for managing sepsis in a critical care unit?",
    "Query 2": "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "Query 3": "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "Query 4": "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "Query 5": "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
}


In [44]:
# --- RAG Response Function ---
def generate_rag_response(user_input, retriever, max_tokens=128, temperature=0, top_p=0.95, top_k=50):
    global qna_system_message, qna_user_message_template
    try:
        relevant_document_chunks = retriever.invoke(user_input)
        context_list = [d.page_content for d in relevant_document_chunks]
        context_for_query = ". ".join(context_list)[:4000]  # Limit for M4
        user_message = qna_user_message_template.replace('{context}', context_for_query).replace('{question}', user_input)
        prompt = f"{qna_system_message}\n{user_message}".strip()
        response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        return response['choices'][0]['text'].strip(), context_for_query
    except ValueError as ve:
        return f"Retrieval error: {ve}", ""
    except Exception as e:
        return f"Sorry, I encountered the following error: {e}", ""

In [45]:
# --- Groundedness and Relevance Evaluation Function ---
def generate_ground_relevance_response(user_input, retriever, max_tokens=10, temperature=0, top_p=0.95, top_k=50):
    global groundedness_rater_system_message, relevance_rater_system_message, user_message_template
    try:
        # Generate RAG response and get context
        answer, context_for_query = generate_rag_response(user_input, retriever, max_tokens=128, temperature=0)
        
        # Groundedness evaluation
        groundedness_prompt = user_message_template.format(
            question=user_input,
            answer=answer,
            context=context_for_query
        )
        groundedness_response = llm(
            prompt=f"{groundedness_rater_system_message}\n{groundedness_prompt}",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        groundedness_rating = groundedness_response['choices'][0]['text'].strip()

        # Relevance evaluation
        relevance_prompt = user_message_template.format(
            question=user_input,
            answer=answer,
            context=context_for_query
        )
        relevance_response = llm(
            prompt=f"{relevance_rater_system_message}\n{relevance_prompt}",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        relevance_rating = relevance_response['choices'][0]['text'].strip()

        return groundedness_rating, relevance_rating, answer, context_for_query
    except Exception as e:
        return f"Error: {e}", f"Error: {e}", "", ""


In [46]:
# --- Fine-tuning Parameters ---
param_combinations = [
    {"chunk_size": 500, "chunk_overlap": 100, "retriever_k": 3, "llm_max_tokens": 128, "llm_temperature": 0},
    {"chunk_size": 750, "chunk_overlap": 150, "retriever_k": 5, "llm_max_tokens": 200, "llm_temperature": 0.1},
    {"chunk_size": 1000, "chunk_overlap": 200, "retriever_k": 3, "llm_max_tokens": 128, "llm_temperature": 0},
    {"chunk_size": 500, "chunk_overlap": 100, "retriever_k": 5, "llm_max_tokens": 200, "llm_temperature": 0.2},
    {"chunk_size": 750, "chunk_overlap": 150, "retriever_k": 3, "llm_max_tokens": 128, "llm_temperature": 0.1}
]


In [47]:
results = {}
evaluation_results = {}

print("\n--- Fine-tuning RAG Parameters ---")
for i, params in enumerate(param_combinations):
    print(f"\n--- Combination {i+1}: {params} ---")
    
    # Chunking (assumes get_data_chunks is defined as in previous code)
    current_chunks = get_data_chunks(
        documents,
        chunk_size=params["chunk_size"],
        chunk_overlap=params["chunk_overlap"],
        split_method="recursive",
        separators=["\n\n", "\n", ". ", " ", ""],
        min_chunk_size=50,
        add_metadata=True
    )
    print(f"  Created {len(current_chunks)} chunks.")
    
    # Embedding
    current_embedded_chunks = []
    batch_size = 50  # Safe for M4
    for j in range(0, len(current_chunks), batch_size):
        chunk_batch = current_chunks[j:j+batch_size]
        try:
            embeddings = embedding_function.embed_documents([chunk.page_content for chunk in chunk_batch])
            for chunk, embedding in zip(chunk_batch, embeddings):
                current_embedded_chunks.append((chunk, embedding))
        except Exception as e:
            print(f"  Error embedding batch {j}: {e}")
    print(f"  Embedded {len(current_embedded_chunks)} chunks.")

    # Vector Database
    persist_directory = f'medical_db_combo_{i+1}'
    if os.path.exists(persist_directory):
        shutil.rmtree(persist_directory, ignore_errors=True)
    os.makedirs(persist_directory, exist_ok=True)
    
    if current_embedded_chunks:
        try:
            current_vector_db = Chroma.from_documents(
                documents=[chunk for chunk, embedding in current_embedded_chunks],
                embedding=embedding_function,
                persist_directory=persist_directory
            )
            current_retriever = current_vector_db.as_retriever(
                search_type="similarity",
                search_kwargs={"k": params["retriever_k"]}
            )
            print(f"  Retriever created with k={params['retriever_k']}")
            
            # Query Execution and Evaluation
            results[f"Combination {i+1}"] = {}
            evaluation_results[f"Combination {i+1}"] = {}
            for query_name, query_text in queries_to_test.items():
                print(f"    Testing Query: {query_name}")
                groundedness_rating, relevance_rating, response_text, context = generate_ground_relevance_response(
                    query_text,
                    current_retriever
                )
                results[f"Combination {i+1}"][query_name] = response_text
                evaluation_results[f"Combination {i+1}"][query_name] = {
                    "groundedness_rating": groundedness_rating,
                    "relevance_rating": relevance_rating,
                    "context": context[:200] + "..." if context else "No context"
                }
                print(f"    Response: {response_text[:200]}...")
                print(f"    Groundedness Rating: {groundedness_rating}")
                print(f"    Relevance Rating: {relevance_rating}")
        except Exception as e:
            print(f"  Error creating vector DB or retriever: {e}")
    else:
        print("  No embedded chunks for vector DB.")



--- Fine-tuning RAG Parameters ---

--- Combination 1: {'chunk_size': 500, 'chunk_overlap': 100, 'retriever_k': 3, 'llm_max_tokens': 128, 'llm_temperature': 0} ---
  Created 34605 chunks.
  Embedded 34605 chunks.
  Retriever created with k=3
    Testing Query: Query 1


Llama.generate: 4 prefix-match hit, remaining 400 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   11237.62 ms /   400 tokens (   28.09 ms per token,    35.59 tokens per second)
llama_perf_context_print:        eval time =   12917.36 ms /    99 runs   (  130.48 ms per token,     7.66 tokens per second)
llama_perf_context_print:       total time =   24178.02 ms /   499 tokens
Llama.generate: 3 prefix-match hit, remaining 517 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   10851.99 ms /   517 tokens (   20.99 ms per token,    47.64 tokens per second)
llama_perf_context_print:        eval time =    1099.45 ms /     9 runs   (  122.16 ms per token,     8.19 tokens per second)
llama_perf_context_print:       total time =   11956.04 ms /   526 tokens
Llama.generate: 9 prefix-match hit, remaining 503 prompt tokens to eval
llama_perf_con

    Response: The protocol for managing sepsis in a critical care unit includes controlling hemorrhage, checking and providing respiratory assistance if necessary, keeping the patient warm, avoiding anything by mou...
    Groundedness Rating: 5
Explanation: The response is
    Relevance Rating: 5
Explanation: The response directly
    Testing Query: Query 2


Llama.generate: 3 prefix-match hit, remaining 469 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   10842.61 ms /   469 tokens (   23.12 ms per token,    43.26 tokens per second)
llama_perf_context_print:        eval time =   15252.35 ms /   127 runs   (  120.10 ms per token,     8.33 tokens per second)
llama_perf_context_print:       total time =   26127.09 ms /   596 tokens
Llama.generate: 3 prefix-match hit, remaining 614 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   12831.00 ms /   614 tokens (   20.90 ms per token,    47.85 tokens per second)
llama_perf_context_print:        eval time =     861.66 ms /     9 runs   (   95.74 ms per token,    10.45 tokens per second)
llama_perf_context_print:       total time =   13694.16 ms /   623 tokens
Llama.generate: 9 prefix-match hit, remaining 600 prompt tokens to eval
llama_perf_con

    Response: The common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, which later shifts to the right lower quadrant. The pain increases with ...
    Groundedness Rating: 5. The response accurately describes the common symptoms
    Relevance Rating: 5
    Testing Query: Query 3


...

    Response: Based on the context provided, the possible causes for sudden patchy hair loss could be alopecia areata. Effective treatments for alopecia areata include topical, intralesional, or systemic corticoste...
    Groundedness Rating: 5. The response accurately identifies alo
    Relevance Rating: 5
    Testing Query: Query 4


Llama.generate: 3 prefix-match hit, remaining 606 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   11599.61 ms /   606 tokens (   19.14 ms per token,    52.24 tokens per second)
llama_perf_context_print:        eval time =   13643.24 ms /   127 runs   (  107.43 ms per token,     9.31 tokens per second)
llama_perf_context_print:       total time =   25266.65 ms /   733 tokens
Llama.generate: 3 prefix-match hit, remaining 751 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   13870.69 ms /   751 tokens (   18.47 ms per token,    54.14 tokens per second)
llama_perf_context_print:        eval time =     801.50 ms /     9 runs   (   89.06 ms per token,    11.23 tokens per second)
llama_perf_context_print:       total time =   14673.36 ms /   760 tokens
Llama.generate: 9 prefix-match hit, remaining 737 prompt tokens to eval
llama_perf_con

    Response: Do not offer opinions or speculate.

Answer: For a person with a brain injury resulting in neurologic deficits, rehabilitation is necessary. This typically involves a team approach with physical, occu...
    Groundedness Rating: 5

The response accurately reflects the context
    Relevance Rating: 5
The response directly addresses the question by
    Testing Query: Query 5


llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   11944.26 ms /   642 tokens (   18.60 ms per token,    53.75 tokens per second)
llama_perf_context_print:        eval time =   13487.38 ms /   127 runs   (  106.20 ms per token,     9.42 tokens per second)
llama_perf_context_print:       total time =   25455.23 ms /   769 tokens
Llama.generate: 3 prefix-match hit, remaining 787 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: prompt eval time =   14470.28 ms /   787 tokens (   18.39 ms per token,    54.39 tokens per second)
llama_perf_context_print:        eval time =     913.15 ms /     9 runs   (  101.46 ms per token,     9.86 tokens per second)
llama_perf_context_print:       total time =   15384.88 ms /   796 tokens
Llama.generate: 9 prefix-match hit, remaining 773 prompt tokens to eval
llama_perf_context_print:        load time =     711.42 ms
llama_perf_context_print: p

    Response: Do not add personal opinions or experiences.

The person with a fractured leg should receive initial treatment in the emergency department. Life-threatening injuries, such as hemorrhagic shock or arte...
    Groundedness Rating: 5
The response is fully supported by the
    Relevance Rating: 5


In [48]:
# --- Compare Results ---
print("\n\n--- Comparison of Results ---")
for query_name, query_text in queries_to_test.items():
    print(f"\n### {query_name}: {query_text}")
    for combo_name, query_results in results.items():
        response_text = query_results.get(query_name, "Response not found")
        eval_data = evaluation_results.get(combo_name, {}).get(query_name, {})
        print(f"\n#### {combo_name} (Chunk Size: {param_combinations[int(combo_name.split(' ')[1])-1]['chunk_size']}, "
              f"Overlap: {param_combinations[int(combo_name.split(' ')[1])-1]['chunk_overlap']}, "
              f"Retriever k: {param_combinations[int(combo_name.split(' ')[1])-1]['retriever_k']}, "
              f"LLM Tokens: {param_combinations[int(combo_name.split(' ')[1])-1]['llm_max_tokens']}, "
              f"LLM Temp: {param_combinations[int(combo_name.split(' ')[1])-1]['llm_temperature']})")
        print(f"Response: {response_text}")
        print(f"Groundedness Rating: {eval_data.get('groundedness_rating', 'N/A')}")
        print(f"Relevance Rating: {eval_data.get('relevance_rating', 'N/A')}")
        print(f"Context Preview: {eval_data.get('context', 'N/A')}")
        print("-" * 50)




--- Comparison of Results ---

### Query 1: What is the protocol for managing sepsis in a critical care unit?

#### Combination 1 (Chunk Size: 500, Overlap: 100, Retriever k: 3, LLM Tokens: 128, LLM Temp: 0)
Response: The protocol for managing sepsis in a critical care unit includes controlling hemorrhage, checking and providing respiratory assistance if necessary, keeping the patient warm, avoiding anything by mouth, draining abscesses, and surgically excising necrotic tissues. Septic foci must be eliminated to prevent further deterioration. Normalization of blood glucose also improves outcome in critically ill patients, even in those not previously known to have diabetes.
Groundedness Rating: 5
Explanation: The response is
Relevance Rating: 5
Explanation: The response directly
Context Preview: 16 - Critical Care Medicine
Chapter 222. Approach to the Critically Ill Patient
Introduction
Critical care medicine specializes in caring for the most seriously ill patients. These patients a

In [49]:
# --- Evaluation Summary ---
print("\n--- Evaluation Results Summary ---")
eval_summary = {}
for combo_name in evaluation_results:
    eval_summary[combo_name] = {}
    for query_name in queries_to_test:
        eval_data = evaluation_results[combo_name].get(query_name, {})
        eval_summary[combo_name][query_name] = {
            "Groundedness": eval_data.get("groundedness_rating", "N/A"),
            "Relevance": eval_data.get("relevance_rating", "N/A")
        }
eval_df = pd.DataFrame.from_dict({(c, q): eval_summary[c][q] for c in eval_summary for q in eval_summary[c]}, orient='index')
print(eval_df)


--- Evaluation Results Summary ---
                                                            Groundedness  \
Combination 1 Query 1                    5\nExplanation: The response is   
              Query 2  5. The response accurately describes the commo...   
              Query 3          5\nThe response accurately identifies alo   
              Query 4   5. The response accurately reflects the context,   
              Query 5  5. The response accurately reflects the contex...   
Combination 2 Query 1  5. The response accurately describes the proto...   
              Query 2          5. All factual claims are fully supported   
              Query 3          5. The response accurately identifies alo   
              Query 4          5. The response is fully supported by the   
              Query 5          5\nAll factual claims are fully supported   
Combination 3 Query 1                5\nThe protocol for managing sepsis   
              Query 2          5. All factual claims